In [1]:
import nltk
nltk.download('sentiwordnet')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\ziyuxiong\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\sentiwordnet.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ziyuxiong\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ziyuxiong\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ziyuxiong\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [1]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import textblob
from textblob import TextBlob
import nltk
from nltk.corpus import sentiwordnet as swn, wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import word_tokenize

In [2]:
!pip install vaderSentiment

   ---------------------------------------- 0.0/126.0 kB ? eta -:--:--
   ------ -------------------------------- 20.5/126.0 kB 330.3 kB/s eta 0:00:01
   ---------------------------------------  122.9/126.0 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 126.0/126.0 kB 1.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# Try different encodings to read the CSV file
# Read easyJet Test Data
try:
    data = pd.read_csv('easyJet_test.csv', encoding='utf-8')
except UnicodeDecodeError:
    data = pd.read_csv('easyJet_test.csv', encoding='ISO-8859-1')

# Display the first few lines of data to ensure that the data is read correctly.
print(data.head())

                                              review real_sentiment
0   Had not flown EasyJet for over a year because...       negative
1     Given the basic fare cost with no hold bagg...       negative
2   My EasyJet flights from Oporto to Ajaccio wer...       positive
3  ? ?EasyJet are scammers. We were never informe...       negative
4    I have been travelling for 5 years, between ...       negative


In [2]:
# Try different encodings to read the CSV file
# Read British Airways Test Data
try:
    data_2 = pd.read_csv('british_test.csv', encoding='utf-8')
except UnicodeDecodeError:
    data_2 = pd.read_csv('british_test.csv', encoding='ISO-8859-1')

# Display the first few lines of data to ensure that the data is read correctly.
print(data_2.head())

                                              review real_sentiment
0   If you can¡¯t fly First Class don¡¯t fly Brit...       negative
1     Singapore to Heathrow. Business class on an...       negative
2     I reported my damaged/ruined suitcase 5 wee...       negative
3  ? ?On March 1st, I flew from Berlin to S?o Pau...       negative
4   The WORST customer experience! British Airway...       negative


In [3]:
analyzer = SentimentIntensityAnalyzer()

# Define VADER sentiment classification function
def vader_sentiment(review):
    sentiment_score = analyzer.polarity_scores(review)
    if sentiment_score['compound'] >= 0.05:
        return 'positive'
    elif sentiment_score['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

In [5]:
# Apply VADER: easyJet
data['vader_sentiment'] = data['review'].apply(vader_sentiment)

# Calculating VADER's evaluation metrics
accuracy_vader = accuracy_score(data['real_sentiment'], data['vader_sentiment'])
precision_vader = precision_score(data['real_sentiment'], data['vader_sentiment'], average='weighted')
recall_vader = recall_score(data['real_sentiment'], data['vader_sentiment'], average='weighted')
f1_vader = f1_score(data['real_sentiment'], data['vader_sentiment'], average='weighted')

print(f"VADER Accuracy: {accuracy_vader}")
print(f"VADER Precision: {precision_vader}")
print(f"VADER Recall: {recall_vader}")
print(f"VADER F1 Score: {f1_vader}")

VADER Accuracy: 0.745
VADER Precision: 0.6956657190239279
VADER Recall: 0.745
VADER F1 Score: 0.7094656546202937


In [4]:
# Apply VADER: British Airways
data_2['vader_sentiment'] = data_2['review'].apply(vader_sentiment)

# Calculating VADER's evaluation metrics
accuracy_vader = accuracy_score(data_2['real_sentiment'], data_2['vader_sentiment'])
precision_vader = precision_score(data_2['real_sentiment'], data_2['vader_sentiment'], average='weighted')
recall_vader = recall_score(data_2['real_sentiment'], data_2['vader_sentiment'], average='weighted')
f1_vader = f1_score(data_2['real_sentiment'], data_2['vader_sentiment'], average='weighted')

print(f"VADER Accuracy: {accuracy_vader}")
print(f"VADER Precision: {precision_vader}")
print(f"VADER Recall: {recall_vader}")
print(f"VADER F1 Score: {f1_vader}")

VADER Accuracy: 0.7
VADER Precision: 0.6873564143853998
VADER Recall: 0.7
VADER F1 Score: 0.6706636892397174


In [5]:
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, SentimentOptions
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [6]:
# Setting up the IBM Watson NLU service
authenticator = IAMAuthenticator('2JQNn0KEKomFqt0Id-cQTkH6z4nHn5KZqTLLUNFaFcuh')
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2022-04-07',
    authenticator=authenticator)
natural_language_understanding.set_service_url('https://api.eu-gb.natural-language-understanding.watson.cloud.ibm.com/instances/d255cf53-8629-4af0-9ecd-560ec0047a3e')

# Define IBM Watson NLU sentiment classification function
def watson_sentiment(review):
    response = natural_language_understanding.analyze(
        text=review,
        features=Features(sentiment=SentimentOptions())
    ).get_result()
    sentiment = response['sentiment']['document']['label']
    return sentiment

In [8]:
# Apply IBM Watson NLU: easyJet
data['watson_sentiment'] = data['review'].apply(watson_sentiment)

# Calculating IBM Watson NLU's evaluation metrics
accuracy_watson = accuracy_score(data['real_sentiment'], data['watson_sentiment'])
precision_watson = precision_score(data['real_sentiment'], data['watson_sentiment'], average='weighted', zero_division=0)
recall_watson = recall_score(data['real_sentiment'], data['watson_sentiment'], average='weighted', zero_division=0)
f1_watson = f1_score(data['real_sentiment'], data['watson_sentiment'], average='weighted', zero_division=0)

print(f"Watson NLU Accuracy: {accuracy_watson}")
print(f"Watson NLU Precision: {precision_watson}")
print(f"Watson NLU Recall: {recall_watson}")
print(f"Watson NLU F1 Score: {f1_watson}")

Watson NLU Accuracy: 0.83
Watson NLU Precision: 0.71834375
Watson NLU Recall: 0.83
Watson NLU F1 Score: 0.7701328753502668


In [7]:
# Apply IBM Watson NLU: British Airways
data_2['watson_sentiment'] = data_2['review'].apply(watson_sentiment)

# Calculating IBM Watson NLU's evaluation metrics
accuracy_watson = accuracy_score(data_2['real_sentiment'], data_2['watson_sentiment'])
precision_watson = precision_score(data_2['real_sentiment'], data_2['watson_sentiment'], average='weighted', zero_division=0)
recall_watson = recall_score(data_2['real_sentiment'], data_2['watson_sentiment'], average='weighted', zero_division=0)
f1_watson = f1_score(data_2['real_sentiment'], data_2['watson_sentiment'], average='weighted', zero_division=0)

print(f"Watson NLU Accuracy: {accuracy_watson}")
print(f"Watson NLU Precision: {precision_watson}")
print(f"Watson NLU Recall: {recall_watson}")
print(f"Watson NLU F1 Score: {f1_watson}")

Watson NLU Accuracy: 0.82
Watson NLU Precision: 0.6927028685381194
Watson NLU Recall: 0.82
Watson NLU F1 Score: 0.7509906592419615


In [8]:
#Define TextBlob function
def textblob_sentiment(text):
    testimonial = TextBlob(text)
    polarity = testimonial.sentiment.polarity
    if polarity > 0:
        return 'positive'
    elif polarity < 0:
        return 'negative'
    else:
        return 'neutral'

In [8]:
#TextBlob: easyJet
data['textblob_sentiment'] = data['review'].apply(textblob_sentiment)


accuracy_textblob = accuracy_score(data['real_sentiment'], data['textblob_sentiment'])
precision_textblob = precision_score(data['real_sentiment'], data['textblob_sentiment'], average='weighted')
recall_textblob = recall_score(data['real_sentiment'], data['textblob_sentiment'], average='weighted')
f1_textblob = f1_score(data['real_sentiment'], data['textblob_sentiment'], average='weighted')

print(f"TextBlob Accuracy: {accuracy_textblob}")
print(f"TextBlob Precision: {precision_textblob}")
print(f"TextBlob Recall: {recall_textblob}")
print(f"TextBlob F1 Score: {f1_textblob}")

TextBlob Accuracy: 0.615
TextBlob Precision: 0.7471906164527666
TextBlob Recall: 0.615
TextBlob F1 Score: 0.6234941576784798


In [9]:
#TextBlob":British Airways
data_2['textblob_sentiment'] = data_2['review'].apply(textblob_sentiment)


accuracy_textblob = accuracy_score(data_2['real_sentiment'], data_2['textblob_sentiment'])
precision_textblob = precision_score(data_2['real_sentiment'], data_2['textblob_sentiment'], average='weighted')
recall_textblob = recall_score(data_2['real_sentiment'], data_2['textblob_sentiment'], average='weighted')
f1_textblob = f1_score(data_2['real_sentiment'], data_2['textblob_sentiment'], average='weighted')

print(f"TextBlob Accuracy: {accuracy_textblob}")
print(f"TextBlob Precision: {precision_textblob}")
print(f"TextBlob Recall: {recall_textblob}")
print(f"TextBlob F1 Score: {f1_textblob}")

TextBlob Accuracy: 0.575
TextBlob Precision: 0.6460233798195242
TextBlob Recall: 0.575
TextBlob F1 Score: 0.564662649846231


In [10]:
#Define SentiWordNet
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def sentiwordnet_sentiment(text):
    sentiment = 0.0
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)
    for word, tag in tagged:
        wn_tag = get_wordnet_pos(tag)
        if wn_tag not in (wordnet.NOUN, wordnet.ADJ, wordnet.ADV, wordnet.VERB):
            continue
        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        if not lemma:
            continue
        synsets = wordnet.synsets(lemma, pos=wn_tag)
        if not synsets:
            continue
        synset = synsets[0]
        swn_synset = swn.senti_synset(synset.name())
        sentiment += swn_synset.pos_score() - swn_synset.neg_score()
    return 'positive' if sentiment > 0 else 'negative' if sentiment < 0 else 'neutral'

In [14]:
#SentiWordNet: easyJet
data['sentiwordnet_sentiment'] = data['review'].apply(sentiwordnet_sentiment)

accuracy_sentiwordnet = accuracy_score(data['real_sentiment'], data['sentiwordnet_sentiment'])
precision_sentiwordnet = precision_score(data['real_sentiment'], data['sentiwordnet_sentiment'], average='weighted')
recall_sentiwordnet = recall_score(data['real_sentiment'], data['sentiwordnet_sentiment'], average='weighted')
f1_sentiwordnet = f1_score(data['real_sentiment'], data['sentiwordnet_sentiment'], average='weighted')

print(f"SentiWordNet Accuracy: {accuracy_sentiwordnet}")
print(f"SentiWordNet Precision: {precision_sentiwordnet}")
print(f"SentiWordNet Recall: {recall_sentiwordnet}")
print(f"SentiWordNet F1 Score: {f1_sentiwordnet}")

SentiWordNet Accuracy: 0.495
SentiWordNet Precision: 0.6513214246695073
SentiWordNet Recall: 0.495
SentiWordNet F1 Score: 0.5133528138528138


In [11]:
#SentiWordNet: British Airways
data_2['sentiwordnet_sentiment'] = data_2['review'].apply(sentiwordnet_sentiment)

accuracy_sentiwordnet = accuracy_score(data_2['real_sentiment'], data_2['sentiwordnet_sentiment'])
precision_sentiwordnet = precision_score(data_2['real_sentiment'], data_2['sentiwordnet_sentiment'], average='weighted')
recall_sentiwordnet = recall_score(data_2['real_sentiment'], data_2['sentiwordnet_sentiment'], average='weighted')
f1_sentiwordnet = f1_score(data_2['real_sentiment'], data_2['sentiwordnet_sentiment'], average='weighted')

print(f"SentiWordNet Accuracy: {accuracy_sentiwordnet}")
print(f"SentiWordNet Precision: {precision_sentiwordnet}")
print(f"SentiWordNet Recall: {recall_sentiwordnet}")
print(f"SentiWordNet F1 Score: {f1_sentiwordnet}")

SentiWordNet Accuracy: 0.52
SentiWordNet Precision: 0.6179798622151563
SentiWordNet Recall: 0.52
SentiWordNet F1 Score: 0.5172602739726028
